In [1]:
from pathlib import Path
import yaml
import os

root_dir = Path("/media") / "sharedData" / "data"
root_name = "glass-a2744"

catalogue_dir = root_dir / "2024_08_16_A2744_v4" / "glass_niriss" / "match_catalogues"

grizli_home_dir = root_dir / "2024_08_16_A2744_v4" / "grizli_home"
grizli_extraction_dir = grizli_home_dir / "Extractions_v2"
os.chdir(grizli_extraction_dir)

In [2]:
from grizli import jwst_utils, multifit
import logging
import shutil

jwst_utils.QUIET_LEVEL = logging.WARNING
jwst_utils.set_quiet_logging(jwst_utils.QUIET_LEVEL)

In [3]:
from astropy.table import Table, join
import numpy as np

cat_v1 = Table.read(catalogue_dir / "classification_v1" / "pyGCG_output_BV.fits")
phot_cat = Table.read(catalogue_dir / "grizli_photz_matched.fits")

In [4]:
# print (np.nansum(np.isin(cat_v1["ID"],phot_cat["NUMBER"])))

cat_v1.remove_columns(["RA","DEC"])

cat_v1["NUMBER"] = cat_v1["ID"].astype(int)
cat_v2 = join(phot_cat, cat_v1, keys="NUMBER", join_type="left")
cat_v2["V1_CLASS"] = 10

cat_v2

NUMBER,NPIX,XMIN,XMAX,YMIN,YMAX,X,Y,X2_IMAGE,Y2_IMAGE,XY_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,FLUX,PEAK,XPEAK,YPEAK,CFLUX,CPEAK,XCPEAK,YCPEAK,X_IMAGE,Y_IMAGE,RA,DEC,X_WORLD,Y_WORLD,FLUX_ISO,FLUXERR_ISO,AREA_ISO,MAG_ISO,KRON_RADIUS,KRON_RCIRC,FLUX_AUTO,FLUXERR_AUTO,BKG_AUTO,FLAG_AUTO,AREA_AUTO,FLUX_RADIUS_FLAG,FLUX_RADIUS_20,FLUX_RADIUS,FLUX_RADIUS_90,TOT_CORR,MAG_AUTO,MAGERR_AUTO,FLUX_APER_0,FLUXERR_APER_0,FLAG_APER_0,BKG_APER_0,MASK_APER_0,FLUX_APER_1,FLUXERR_APER_1,FLAG_APER_1,BKG_APER_1,MASK_APER_1,FLUX_APER_2,FLUXERR_APER_2,FLAG_APER_2,BKG_APER_2,MASK_APER_2,FLUX_APER_3,FLUXERR_APER_3,FLAG_APER_3,BKG_APER_3,MASK_APER_3,FLUX_APER_4,FLUXERR_APER_4,FLAG_APER_4,BKG_APER_4,MASK_APER_4,FLUX_APER_5,FLUXERR_APER_5,FLAG_APER_5,BKG_APER_5,MASK_APER_5,FLUX_APER_6,FLUXERR_APER_6,FLAG_APER_6,BKG_APER_6,MASK_APER_6,id_astrodeep,ra,dec,zspec,EAzY_eazy_v13,EAzY_Larson,EAzY_LarsonLyaRed,flag,id_alt,n_lines_detected,id_astrodeep_zphot,zphot,ID,SEG_ID,"F115W,72.0_QUALITY","F115W,72.0_COVERAGE","F150W,72.0_QUALITY","F150W,72.0_COVERAGE","F200W,72.0_QUALITY","F200W,72.0_COVERAGE","F115W,341.0_QUALITY","F115W,341.0_COVERAGE","F150W,341.0_QUALITY","F150W,341.0_COVERAGE","F200W,341.0_QUALITY","F200W,341.0_COVERAGE",GRIZLI_REDSHIFT,ESTIMATED_REDSHIFT,UNRELIABLE_REDSHIFT,TENTATIVE_REDSHIFT,BAD_SEG_MAP,COMMENTS,V1_CLASS
,pix2,,,,,,,pix2,pix2,pix2,pix,pix,deg,1 / pix2,1 / pix2,1 / pix2,uJy,uJy,,,uJy,uJy,,,,,deg,deg,deg,deg,uJy,uJy,,uJy,pix,pix2,uJy,uJy,uJy,,pix,,,,,,uJy,uJy,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,int64,float64,float64,float64,float32,float32,float32,int64,int64,int64,int64,float64,bytes4,int64,bytes8,float64,bytes8,float64,bytes8,float64,bytes8,float64,bytes9,float64,bytes8,float64,float64,float64,bool,bool,bool,bytes152,int64
1,4437.0,4751,4838,249,326,4812.394814160201,272.58716944015896,161.86670879464415,45.31700863706545,16.99989651917582,12.81778787030681,6.548895444540644,0.14192129225511604,0.0064313028510793666,0.022971812506493813,-0.004825185343871344,42.3196762041362,0.5002276054571878,4818,273,42.3196762041362,0.5002276054571878,4818,273,4813.394814160201,273.58716944015896,3.580726811401407,-30.41884945789135,3.580726811401407,-30.41884945789135,42.314310344362504,0.01896852576138402,4428,19.833781832593232,2.7296609742478113,25.009170022756557,47.8844554717923,0.01589969386814892,-0.02728534352236423,32,1964.936096474265,32,4.837337250863512,9.690678715425213,31.099771196660793,1.1172326635864773,19.57915455643092,0.0003605110073223929,13.775889265431,0.00880093371767937,0,-0.0014799562498162696,0.0,20.895457702405487,0.010993693054033528,0,-0.0028643425732287515,0.0,27.423026581252955,0.013072030300571002,0,-0.005642801761808256,0.0,33.50075212359719,0.015687019183477122,0,-0.011563782442188314,0.0,37.1408552196513,0.021225927785926836,32,-0.01675335065237328,27.491260800197125,43.907226408413756,0.033296290106755055,32,-0.02668123800707294,234.0386037077305,74.85119835209743,0.13252350415630232,32,-0.12042262499067907,3414.1468288836995,5504,3.580686,-30.418848,0.293,0.293,0.293,0.293,212,--,--,5504,0.293,1,1,Good,0.0,Good,0.0,Good,0.0,Unusable,0.8461538461538461,Unusable,0.0,Unusable,0.0,0.28800000000000003,0.28800000000000003,False,False,False,--,10
2,151.0,4818,4833,319,

In [5]:
from itertools import product
from numpy.lib.recfunctions import structured_to_unstructured

filters = ["F115W","F150W","F200W"]
pas = ["72.0","341.0"]

beam_names = [f"{a},{b}" for b, a in product(pas, filters)]
coverage_names = [f"{b}_COVERAGE" for b in beam_names]
quality_names = [f"{b}_QUALITY" for b in beam_names]
new_names = [f"{b}_USE" for b in beam_names]

missing_ids = ~np.isin(cat_v2["NUMBER"],cat_v1["ID"])

cat_v2["V1_CLASS"][missing_ids] = 0

for q in new_names:
    cat_v2[q] = 1
    cat_v2[q][missing_ids] = 0

cat_v2


NUMBER,NPIX,XMIN,XMAX,YMIN,YMAX,X,Y,X2_IMAGE,Y2_IMAGE,XY_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,FLUX,PEAK,XPEAK,YPEAK,CFLUX,CPEAK,XCPEAK,YCPEAK,X_IMAGE,Y_IMAGE,RA,DEC,X_WORLD,Y_WORLD,FLUX_ISO,FLUXERR_ISO,AREA_ISO,MAG_ISO,KRON_RADIUS,KRON_RCIRC,FLUX_AUTO,FLUXERR_AUTO,BKG_AUTO,FLAG_AUTO,AREA_AUTO,FLUX_RADIUS_FLAG,FLUX_RADIUS_20,FLUX_RADIUS,FLUX_RADIUS_90,TOT_CORR,MAG_AUTO,MAGERR_AUTO,FLUX_APER_0,FLUXERR_APER_0,FLAG_APER_0,BKG_APER_0,MASK_APER_0,FLUX_APER_1,FLUXERR_APER_1,FLAG_APER_1,BKG_APER_1,MASK_APER_1,FLUX_APER_2,FLUXERR_APER_2,FLAG_APER_2,BKG_APER_2,MASK_APER_2,FLUX_APER_3,FLUXERR_APER_3,FLAG_APER_3,BKG_APER_3,MASK_APER_3,FLUX_APER_4,FLUXERR_APER_4,FLAG_APER_4,BKG_APER_4,MASK_APER_4,FLUX_APER_5,FLUXERR_APER_5,FLAG_APER_5,BKG_APER_5,MASK_APER_5,FLUX_APER_6,FLUXERR_APER_6,FLAG_APER_6,BKG_APER_6,MASK_APER_6,id_astrodeep,ra,dec,zspec,EAzY_eazy_v13,EAzY_Larson,EAzY_LarsonLyaRed,flag,id_alt,n_lines_detected,id_astrodeep_zphot,zphot,ID,SEG_ID,"F115W,72.0_QUALITY","F115W,72.0_COVERAGE","F150W,72.0_QUALITY","F150W,72.0_COVERAGE","F200W,72.0_QUALITY","F200W,72.0_COVERAGE","F115W,341.0_QUALITY","F115W,341.0_COVERAGE","F150W,341.0_QUALITY","F150W,341.0_COVERAGE","F200W,341.0_QUALITY","F200W,341.0_COVERAGE",GRIZLI_REDSHIFT,ESTIMATED_REDSHIFT,UNRELIABLE_REDSHIFT,TENTATIVE_REDSHIFT,BAD_SEG_MAP,COMMENTS,V1_CLASS,"F115W,72.0_USE","F150W,72.0_USE","F200W,72.0_USE","F115W,341.0_USE","F150W,341.0_USE","F200W,341.0_USE"
,pix2,,,,,,,pix2,pix2,pix2,pix,pix,deg,1 / pix2,1 / pix2,1 / pix2,uJy,uJy,,,uJy,uJy,,,,,deg,deg,deg,deg,uJy,uJy,,uJy,pix,pix2,uJy,uJy,uJy,,pix,,,,,,uJy,uJy,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,int64,float64,float64,float64,float32,float32,float32,int64,int64,int64,int64,float64,bytes4,int64,bytes8,float64,bytes8,float64,bytes8,float64,bytes8,float64,bytes9,float64,bytes8,float64,float64,float64,bool,bool,bool,bytes152,int64,int64,int64,int64,int64,int64,int64
1,4437.0,4751,4838,249,326,4812.394814160201,272.58716944015896,161.86670879464415,45.31700863706545,16.99989651917582,12.81778787030681,6.548895444540644,0.14192129225511604,0.0064313028510793666,0.022971812506493813,-0.004825185343871344,42.3196762041362,0.5002276054571878,4818,273,42.3196762041362,0.5002276054571878,4818,273,4813.394814160201,273.58716944015896,3.580726811401407,-30.41884945789135,3.580726811401407,-30.41884945789135,42.314310344362504,0.01896852576138402,4428,19.833781832593232,2.7296609742478113,25.009170022756557,47.8844554717923,0.01589969386814892,-0.02728534352236423,32,1964.936096474265,32,4.837337250863512,9.690678715425213,31.099771196660793,1.1172326635864773,19.57915455643092,0.0003605110073223929,13.775889265431,0.00880093371767937,0,-0.0014799562498162696,0.0,20.895457702405487,0.010993693054033528,0,-0.0028643425732287515,0.0,27.423026581252955,0.013072030300571002,0,-0.005642801761808256,0.0,33.50075212359719,0.015687019183477122,0,-0.011563782442188314,0.0,37.1408552196513,0.021225927785926836,32,-0.01675335065237328,27.491260800197125,43.907226408413756,0.033296290106755055,32,-0.02668123800707294,234.0386037077305,74.85119835209743,0.13252350415630232,32,-0.12042262499067907,3414.1468288836995,5504,3.580686,-30.418848,0.293,0.293,0.293,0.293,212,--,--,5504,0.293,1,1,Good,0.0,Good,0.0,Go

In [6]:
misplaced_ids = np.array([1,8,10,17,52,63,77,99,104,115,120,174])

for i_p, pa in enumerate(pas):
    unusable_f150w_flag = (
        (cat_v2[f"F150W,{pa}_QUALITY"]=="Unusable")
        & (cat_v2[f"F150W,{pa}_COVERAGE"]!=0)
        & ((cat_v2["NUMBER"].astype(int)<260) & (~np.isin(cat_v2["NUMBER"].astype(int), misplaced_ids))) 

    )
    for filt in filters:
        cat_v2[f"{filt},{pa}_USE"][unusable_f150w_flag] = 0

    unusable_f115w_flag = (
        (cat_v2[f"F115W,{pa}_QUALITY"]=="Unusable")
        & (cat_v2[f"F115W,{pa}_COVERAGE"]!=0)
        & ((cat_v2["NUMBER"].astype(int)<260) & (~np.isin(cat_v2["NUMBER"].astype(int), misplaced_ids)))
    )
    for filt in filters:
        cat_v2[f"{filt},{pa}_USE"][unusable_f115w_flag] = 0
        # cat_v2[f"{filt},{pa}_USE"][np.isin(cat_v2["NUMBER"],cat_v1["ID"][unusable_f115w_flag])] = 0

# cat_v2["ESTIMATED_REDSHIFT"] = np.nan
# cat_v2["ESTIMATED_REDSHIFT"][np.isin(cat_v2["NUMBER"],cat_v1["ID"])] = cat_v1["ESTIMATED_REDSHIFT"]

# print (np.where(cat_v2["NUMBER"]==cat_v1["ID"]))
# a,b,v1_to_v2_idx = np.intersect1d(cat_v2["NUMBER"],cat_v1["ID"], return_indices=True)
# cat_v2["ESTIMATED_REDSHIFT"][b] = cat_v1["ESTIMATED_REDSHIFT"]
cat_v2


NUMBER,NPIX,XMIN,XMAX,YMIN,YMAX,X,Y,X2_IMAGE,Y2_IMAGE,XY_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,FLUX,PEAK,XPEAK,YPEAK,CFLUX,CPEAK,XCPEAK,YCPEAK,X_IMAGE,Y_IMAGE,RA,DEC,X_WORLD,Y_WORLD,FLUX_ISO,FLUXERR_ISO,AREA_ISO,MAG_ISO,KRON_RADIUS,KRON_RCIRC,FLUX_AUTO,FLUXERR_AUTO,BKG_AUTO,FLAG_AUTO,AREA_AUTO,FLUX_RADIUS_FLAG,FLUX_RADIUS_20,FLUX_RADIUS,FLUX_RADIUS_90,TOT_CORR,MAG_AUTO,MAGERR_AUTO,FLUX_APER_0,FLUXERR_APER_0,FLAG_APER_0,BKG_APER_0,MASK_APER_0,FLUX_APER_1,FLUXERR_APER_1,FLAG_APER_1,BKG_APER_1,MASK_APER_1,FLUX_APER_2,FLUXERR_APER_2,FLAG_APER_2,BKG_APER_2,MASK_APER_2,FLUX_APER_3,FLUXERR_APER_3,FLAG_APER_3,BKG_APER_3,MASK_APER_3,FLUX_APER_4,FLUXERR_APER_4,FLAG_APER_4,BKG_APER_4,MASK_APER_4,FLUX_APER_5,FLUXERR_APER_5,FLAG_APER_5,BKG_APER_5,MASK_APER_5,FLUX_APER_6,FLUXERR_APER_6,FLAG_APER_6,BKG_APER_6,MASK_APER_6,id_astrodeep,ra,dec,zspec,EAzY_eazy_v13,EAzY_Larson,EAzY_LarsonLyaRed,flag,id_alt,n_lines_detected,id_astrodeep_zphot,zphot,ID,SEG_ID,"F115W,72.0_QUALITY","F115W,72.0_COVERAGE","F150W,72.0_QUALITY","F150W,72.0_COVERAGE","F200W,72.0_QUALITY","F200W,72.0_COVERAGE","F115W,341.0_QUALITY","F115W,341.0_COVERAGE","F150W,341.0_QUALITY","F150W,341.0_COVERAGE","F200W,341.0_QUALITY","F200W,341.0_COVERAGE",GRIZLI_REDSHIFT,ESTIMATED_REDSHIFT,UNRELIABLE_REDSHIFT,TENTATIVE_REDSHIFT,BAD_SEG_MAP,COMMENTS,V1_CLASS,"F115W,72.0_USE","F150W,72.0_USE","F200W,72.0_USE","F115W,341.0_USE","F150W,341.0_USE","F200W,341.0_USE"
,pix2,,,,,,,pix2,pix2,pix2,pix,pix,deg,1 / pix2,1 / pix2,1 / pix2,uJy,uJy,,,uJy,uJy,,,,,deg,deg,deg,deg,uJy,uJy,,uJy,pix,pix2,uJy,uJy,uJy,,pix,,,,,,uJy,uJy,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,int64,float64,float64,float64,float32,float32,float32,int64,int64,int64,int64,float64,bytes4,int64,bytes8,float64,bytes8,float64,bytes8,float64,bytes8,float64,bytes9,float64,bytes8,float64,float64,float64,bool,bool,bool,bytes152,int64,int64,int64,int64,int64,int64,int64
1,4437.0,4751,4838,249,326,4812.394814160201,272.58716944015896,161.86670879464415,45.31700863706545,16.99989651917582,12.81778787030681,6.548895444540644,0.14192129225511604,0.0064313028510793666,0.022971812506493813,-0.004825185343871344,42.3196762041362,0.5002276054571878,4818,273,42.3196762041362,0.5002276054571878,4818,273,4813.394814160201,273.58716944015896,3.580726811401407,-30.41884945789135,3.580726811401407,-30.41884945789135,42.314310344362504,0.01896852576138402,4428,19.833781832593232,2.7296609742478113,25.009170022756557,47.8844554717923,0.01589969386814892,-0.02728534352236423,32,1964.936096474265,32,4.837337250863512,9.690678715425213,31.099771196660793,1.1172326635864773,19.57915455643092,0.0003605110073223929,13.775889265431,0.00880093371767937,0,-0.0014799562498162696,0.0,20.895457702405487,0.010993693054033528,0,-0.0028643425732287515,0.0,27.423026581252955,0.013072030300571002,0,-0.005642801761808256,0.0,33.50075212359719,0.015687019183477122,0,-0.011563782442188314,0.0,37.1408552196513,0.021225927785926836,32,-0.01675335065237328,27.491260800197125,43.907226408413756,0.033296290106755055,32,-0.02668123800707294,234.0386037077305,74.85119835209743,0.13252350415630232,32,-0.12042262499067907,3414.1468288836995,5504,3.580686,-30.418848,0.293,0.293,0.293,0.293,212,--,--,5504,0.293,1,1,Good,0.0,Good,0.0,Go

In [7]:
for i_p, pa in enumerate(pas):
    for filt in filters:
        unusable_flag = (
            ((cat_v2["NUMBER"].astype(int)>=260) | (np.isin(cat_v2["NUMBER"].astype(int), misplaced_ids))) &
            (
                (cat_v2[f"{filt},{pa}_QUALITY"]=="Unusable")
                | (cat_v2[f"{filt},{pa}_COVERAGE"]==0)
            )
        )
        cat_v2[f"{filt},{pa}_USE"][unusable_flag] = 0
        
cat_v2

NUMBER,NPIX,XMIN,XMAX,YMIN,YMAX,X,Y,X2_IMAGE,Y2_IMAGE,XY_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,FLUX,PEAK,XPEAK,YPEAK,CFLUX,CPEAK,XCPEAK,YCPEAK,X_IMAGE,Y_IMAGE,RA,DEC,X_WORLD,Y_WORLD,FLUX_ISO,FLUXERR_ISO,AREA_ISO,MAG_ISO,KRON_RADIUS,KRON_RCIRC,FLUX_AUTO,FLUXERR_AUTO,BKG_AUTO,FLAG_AUTO,AREA_AUTO,FLUX_RADIUS_FLAG,FLUX_RADIUS_20,FLUX_RADIUS,FLUX_RADIUS_90,TOT_CORR,MAG_AUTO,MAGERR_AUTO,FLUX_APER_0,FLUXERR_APER_0,FLAG_APER_0,BKG_APER_0,MASK_APER_0,FLUX_APER_1,FLUXERR_APER_1,FLAG_APER_1,BKG_APER_1,MASK_APER_1,FLUX_APER_2,FLUXERR_APER_2,FLAG_APER_2,BKG_APER_2,MASK_APER_2,FLUX_APER_3,FLUXERR_APER_3,FLAG_APER_3,BKG_APER_3,MASK_APER_3,FLUX_APER_4,FLUXERR_APER_4,FLAG_APER_4,BKG_APER_4,MASK_APER_4,FLUX_APER_5,FLUXERR_APER_5,FLAG_APER_5,BKG_APER_5,MASK_APER_5,FLUX_APER_6,FLUXERR_APER_6,FLAG_APER_6,BKG_APER_6,MASK_APER_6,id_astrodeep,ra,dec,zspec,EAzY_eazy_v13,EAzY_Larson,EAzY_LarsonLyaRed,flag,id_alt,n_lines_detected,id_astrodeep_zphot,zphot,ID,SEG_ID,"F115W,72.0_QUALITY","F115W,72.0_COVERAGE","F150W,72.0_QUALITY","F150W,72.0_COVERAGE","F200W,72.0_QUALITY","F200W,72.0_COVERAGE","F115W,341.0_QUALITY","F115W,341.0_COVERAGE","F150W,341.0_QUALITY","F150W,341.0_COVERAGE","F200W,341.0_QUALITY","F200W,341.0_COVERAGE",GRIZLI_REDSHIFT,ESTIMATED_REDSHIFT,UNRELIABLE_REDSHIFT,TENTATIVE_REDSHIFT,BAD_SEG_MAP,COMMENTS,V1_CLASS,"F115W,72.0_USE","F150W,72.0_USE","F200W,72.0_USE","F115W,341.0_USE","F150W,341.0_USE","F200W,341.0_USE"
,pix2,,,,,,,pix2,pix2,pix2,pix,pix,deg,1 / pix2,1 / pix2,1 / pix2,uJy,uJy,,,uJy,uJy,,,,,deg,deg,deg,deg,uJy,uJy,,uJy,pix,pix2,uJy,uJy,uJy,,pix,,,,,,uJy,uJy,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,int64,float64,float64,float64,float32,float32,float32,int64,int64,int64,int64,float64,bytes4,int64,bytes8,float64,bytes8,float64,bytes8,float64,bytes8,float64,bytes9,float64,bytes8,float64,float64,float64,bool,bool,bool,bytes152,int64,int64,int64,int64,int64,int64,int64
1,4437.0,4751,4838,249,326,4812.394814160201,272.58716944015896,161.86670879464415,45.31700863706545,16.99989651917582,12.81778787030681,6.548895444540644,0.14192129225511604,0.0064313028510793666,0.022971812506493813,-0.004825185343871344,42.3196762041362,0.5002276054571878,4818,273,42.3196762041362,0.5002276054571878,4818,273,4813.394814160201,273.58716944015896,3.580726811401407,-30.41884945789135,3.580726811401407,-30.41884945789135,42.314310344362504,0.01896852576138402,4428,19.833781832593232,2.7296609742478113,25.009170022756557,47.8844554717923,0.01589969386814892,-0.02728534352236423,32,1964.936096474265,32,4.837337250863512,9.690678715425213,31.099771196660793,1.1172326635864773,19.57915455643092,0.0003605110073223929,13.775889265431,0.00880093371767937,0,-0.0014799562498162696,0.0,20.895457702405487,0.010993693054033528,0,-0.0028643425732287515,0.0,27.423026581252955,0.013072030300571002,0,-0.005642801761808256,0.0,33.50075212359719,0.015687019183477122,0,-0.011563782442188314,0.0,37.1408552196513,0.021225927785926836,32,-0.01675335065237328,27.491260800197125,43.907226408413756,0.033296290106755055,32,-0.02668123800707294,234.0386037077305,74.85119835209743,0.13252350415630232,32,-0.12042262499067907,3414.1468288836995,5504,3.580686,-30.418848,0.293,0.293,0.293,0.293,212,--,--,5504,0.293,1,1,Good,0.0,Good,0.0,Go

In [8]:
# V1_CLASS:
# 0 - Extraction not possible
# 1 - Unusable in all beams
# 2 - Star
# 3 - Bad seg map
# 4 - Some bad beams, re-extract
# 5 - No bad beams, re-extract
# 6 - No bad beams, specz ~= estimated z (don't re-extract)

for row in cat_v2[~cat_v2["COMMENTS"].mask]:
    if "star" in row["COMMENTS"]:
        cat_v2["V1_CLASS"][np.isin(cat_v2["NUMBER"],int(row["ID"]))] = 2
    elif "unusable" in row["COMMENTS"]:
        print (row)
        if "72" in row["COMMENTS"]:
            for filt in filters:
                cat_v2[f"{filt},72.0_USE"][np.isin(cat_v2["NUMBER"],int(row["ID"]))] = 0
        elif "341" in row["COMMENTS"]:
            for filt in filters:
                cat_v2[f"{filt},341.0_USE"][np.isin(cat_v2["NUMBER"],int(row["ID"]))] = 0
    elif "remove" in row["COMMENTS"]:
        for pa in pas:
            for filt in filters:
                cat_v2[f"{filt},{pa}_USE"][np.isin(cat_v2["NUMBER"],int(row["ID"]))] = 0
    else:
        pass
        # print (row["ID"],row["COMMENTS"])

cat_v2["V1_CLASS"][
    np.all(structured_to_unstructured(np.array(cat_v2[new_names]))==0, axis=1)
    & (cat_v2["V1_CLASS"]==10)
] = 1

cat_v2["REDSHIFT_USE"] = cat_v2["ESTIMATED_REDSHIFT"]

cat_v2

# print (np.unique(cat_v2["V1_CLASS"], return_counts=True))

NUMBER  NPIX XMIN XMAX YMIN YMAX         X                 Y               X2_IMAGE           Y2_IMAGE          XY_IMAGE           A_IMAGE           B_IMAGE          THETA_IMAGE          CXX_IMAGE           CYY_IMAGE           CXY_IMAGE               FLUX                 PEAK         XPEAK YPEAK        CFLUX                CPEAK         XCPEAK YCPEAK      X_IMAGE           Y_IMAGE               RA                DEC              X_WORLD            Y_WORLD            FLUX_ISO           FLUXERR_ISO      AREA_ISO      MAG_ISO       KRON_RADIUS     KRON_RCIRC          FLUX_AUTO           FLUXERR_AUTO            BKG_AUTO       FLAG_AUTO     AREA_AUTO      FLUX_RADIUS_FLAG   FLUX_RADIUS_20     FLUX_RADIUS      FLUX_RADIUS_90        TOT_CORR           MAG_AUTO          MAGERR_AUTO         FLUX_APER_0         FLUXERR_APER_0    FLAG_APER_0       BKG_APER_0       MASK_APER_0     FLUX_APER_1          FLUXERR_APER_1    FLAG_APER_1       BKG_APER_1       MASK_APER_1     FLUX_APER_2        FLUXERR_A

NUMBER,NPIX,XMIN,XMAX,YMIN,YMAX,X,Y,X2_IMAGE,Y2_IMAGE,XY_IMAGE,A_IMAGE,B_IMAGE,THETA_IMAGE,CXX_IMAGE,CYY_IMAGE,CXY_IMAGE,FLUX,PEAK,XPEAK,YPEAK,CFLUX,CPEAK,XCPEAK,YCPEAK,X_IMAGE,Y_IMAGE,RA,DEC,X_WORLD,Y_WORLD,FLUX_ISO,FLUXERR_ISO,AREA_ISO,MAG_ISO,KRON_RADIUS,KRON_RCIRC,FLUX_AUTO,FLUXERR_AUTO,BKG_AUTO,FLAG_AUTO,AREA_AUTO,FLUX_RADIUS_FLAG,FLUX_RADIUS_20,FLUX_RADIUS,FLUX_RADIUS_90,TOT_CORR,MAG_AUTO,MAGERR_AUTO,FLUX_APER_0,FLUXERR_APER_0,FLAG_APER_0,BKG_APER_0,MASK_APER_0,FLUX_APER_1,FLUXERR_APER_1,FLAG_APER_1,BKG_APER_1,MASK_APER_1,FLUX_APER_2,FLUXERR_APER_2,FLAG_APER_2,BKG_APER_2,MASK_APER_2,FLUX_APER_3,FLUXERR_APER_3,FLAG_APER_3,BKG_APER_3,MASK_APER_3,FLUX_APER_4,FLUXERR_APER_4,FLAG_APER_4,BKG_APER_4,MASK_APER_4,FLUX_APER_5,FLUXERR_APER_5,FLAG_APER_5,BKG_APER_5,MASK_APER_5,FLUX_APER_6,FLUXERR_APER_6,FLAG_APER_6,BKG_APER_6,MASK_APER_6,id_astrodeep,ra,dec,zspec,EAzY_eazy_v13,EAzY_Larson,EAzY_LarsonLyaRed,flag,id_alt,n_lines_detected,id_astrodeep_zphot,zphot,ID,SEG_ID,"F115W,72.0_QUALITY","F115W,72.0_COVERAGE","F150W,72.0_QUALITY","F150W,72.0_COVERAGE","F200W,72.0_QUALITY","F200W,72.0_COVERAGE","F115W,341.0_QUALITY","F115W,341.0_COVERAGE","F150W,341.0_QUALITY","F150W,341.0_COVERAGE","F200W,341.0_QUALITY","F200W,341.0_COVERAGE",GRIZLI_REDSHIFT,ESTIMATED_REDSHIFT,UNRELIABLE_REDSHIFT,TENTATIVE_REDSHIFT,BAD_SEG_MAP,COMMENTS,V1_CLASS,"F115W,72.0_USE","F150W,72.0_USE","F200W,72.0_USE","F115W,341.0_USE","F150W,341.0_USE","F200W,341.0_USE",REDSHIFT_USE
,pix2,,,,,,,pix2,pix2,pix2,pix,pix,deg,1 / pix2,1 / pix2,1 / pix2,uJy,uJy,,,uJy,uJy,,,,,deg,deg,deg,deg,uJy,uJy,,uJy,pix,pix2,uJy,uJy,uJy,,pix,,,,,,uJy,uJy,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,uJy,uJy,,uJy,,,deg,deg,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
int64,float64,int64,int64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,int64,float64,float64,int64,int64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,float64,float64,float64,float64,float64,int64,float64,int16,float64,float64,float64,float64,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,float64,float64,int16,float64,float64,int64,float64,float64,float64,float32,float32,float32,int64,int64,int64,int64,float64,bytes4,int64,bytes8,float64,bytes8,float64,bytes8,float64,bytes8,float64,bytes9,float64,bytes8,float64,float64,float64,bool,bool,bool,bytes152,int64,int64,int64,int64,int64,int64,int64,float64
1,4437.0,4751,4838,249,326,4812.394814160201,272.58716944015896,161.86670879464415,45.31700863706545,16.99989651917582,12.81778787030681,6.548895444540644,0.14192129225511604,0.0064313028510793666,0.022971812506493813,-0.004825185343871344,42.3196762041362,0.5002276054571878,4818,273,42.3196762041362,0.5002276054571878,4818,273,4813.394814160201,273.58716944015896,3.580726811401407,-30.41884945789135,3.580726811401407,-30.41884945789135,42.314310344362504,0.01896852576138402,4428,19.833781832593232,2.7296609742478113,25.009170022756557,47.8844554717923,0.01589969386814892,-0.02728534352236423,32,1964.936096474265,32,4.837337250863512,9.690678715425213,31.099771196660793,1.1172326635864773,19.57915455643092,0.0003605110073223929,13.775889265431,0.00880093371767937,0,-0.0014799562498162696,0.0,20.895457702405487,0.010993693054033528,0,-0.0028643425732287515,0.0,27.423026581252955,0.013072030300571002,0,-0.005642801761808256,0.0,33.50075212359719,0.015687019183477122,0,-0.011563782442188314,0.0,37.1408552196513,0.021225927785926836,32,-0.01675335065237328,27.491260800197125,43.907226408413756,0.033296290106755055,32,-0.02668123800707294,234.0386037077305,74.85119835209743,0.13252350415630232,32,-0.12042262499067907,3414.1468288836995,5504,3.580686,-30.418848,0.293,0.293,0.293,0.293,212,--,--,5504,0.293,1,

In [9]:
cat_v2["V1_CLASS"][(cat_v2["BAD_SEG_MAP"] == True) & (cat_v2["V1_CLASS"] == 10)] = 3


cat_v2["V1_CLASS"][
    np.any(structured_to_unstructured(np.array(cat_v2[new_names])) == 0, axis=1)
    & (cat_v2["V1_CLASS"] == 10)
] = 4

cat_v2["V1_CLASS"][
    np.all(structured_to_unstructured(np.array(cat_v2[new_names])) == 1, axis=1)
    & (cat_v2["V1_CLASS"] == 10)
] = 5


cat_v2["V1_CLASS"][
    (cat_v2["V1_CLASS"] == 5)
    # (cat_v2["ESTIMATED_REDSHIFT"] == cat_v2["GRIZLI_REDSHIFT"])
    & (cat_v2["zspec"] > 0)
    & (np.abs(cat_v2["ESTIMATED_REDSHIFT"]/cat_v2["GRIZLI_REDSHIFT"]-1)<=0.005)
    # & (~np.any(structured_to_unstructured(np.array(cat_v2[new_names])) == 0))
] = 6

# print(np.nansum(cat_v2["zspec"] > 0))

# print (np.abs(cat_v2["ESTIMATED_REDSHIFT"]/cat_v2["GRIZLI_REDSHIFT"])[cat_v2["V1_CLASS"]==6])
# print (quality_names)
# print (np.nansum(
#     (cat_v1[quality_names[1]]=="Unusable") &
#     (cat_v1[coverage_names[1]]!=0)
# ))

# print (np.unique(cat_v2["V1_CLASS"], return_counts=True))

In [10]:
print("Statistics:")
print(f"Total: {len(cat_v2)}")
print(
    f"Usable in at least one beam: {np.nansum(~np.all(structured_to_unstructured(np.array(cat_v2[new_names]))==0, axis=1))}"
)
print(
    f"Usable in all beams: {np.nansum(np.all(structured_to_unstructured(np.array(cat_v2[new_names]))==1, axis=1))}"
)
print("")
print(f"No extraction possible: {np.nansum(cat_v2["V1_CLASS"]==0)}")
print(f"Unusable in all orientations: {np.nansum(cat_v2["V1_CLASS"]==1)}")
print(f"Star: {np.nansum(cat_v2["V1_CLASS"]==2)}")
print(f"Bad seg map: {np.nansum(cat_v2["V1_CLASS"]==3)}")
print(f"Unusable in some orientations: {np.nansum(cat_v2["V1_CLASS"]==4)}")
print(f"Good in all orientations: {np.nansum(cat_v2["V1_CLASS"]==5)}")
print(f"Good in all orientations, and specz: {np.nansum(cat_v2["V1_CLASS"]==6)}")
print("")

use_cat = cat_v2[cat_v2["V1_CLASS"] >= 4]

highz_cat = use_cat[
    ((use_cat["REDSHIFT_USE"] >= 6) | (use_cat["UNRELIABLE_REDSHIFT"] == True) )
    & (np.isfinite(use_cat["zphot"]))
]

lowz_cat = use_cat[
    ((use_cat["REDSHIFT_USE"] < 6) & (use_cat["UNRELIABLE_REDSHIFT"] == False))
    | (~np.isfinite(use_cat["zphot"]))
]

print(f"Number of z>=6: {len(highz_cat)}")
print(f"Number of z<6: {len(lowz_cat)}")
print("")
print(
    f"Number of z>=6 and not unreliable: ",
    np.nansum((highz_cat["UNRELIABLE_REDSHIFT"] == False) | (highz_cat["TENTATIVE_REDSHIFT"]==True))
)
print (highz_cat[highz_cat["UNRELIABLE_REDSHIFT"] == False])
print(
    f"Number of z<6 and tentative: ",
    np.nansum(lowz_cat["TENTATIVE_REDSHIFT"] == True)
)
print (lowz_cat[lowz_cat["TENTATIVE_REDSHIFT"] == True])

cat_out_path = catalogue_dir / "classification_v1" / "compiled_catalogue_v1.fits"
if not cat_out_path.is_file():
    cat_v2.write(catalogue_dir / "classification_v1" / "compiled_catalogue_v1.fits")

Statistics:
Total: 3493
Usable in at least one beam: 2413
Usable in all beams: 1005

No extraction possible: 511
Unusable in all orientations: 565
Star: 9
Bad seg map: 368
Unusable in some orientations: 1173
Good in all orientations: 678
Good in all orientations, and specz: 189

Number of z>=6: 1016
Number of z<6: 1024

Number of z>=6 and not unreliable:  52
NUMBER  NPIX XMIN XMAX ... F150W,341.0_USE F200W,341.0_USE    REDSHIFT_USE  
        pix2           ...                                                  
------ ----- ---- ---- ... --------------- --------------- -----------------
   151  31.0 2970 2976 ...               1               1 6.682500000000001
   283  68.0 2501 2510 ...               1               0 6.018746214754627
   302 237.0 2479 2497 ...               1               1 7.439370541401006
   314 160.0 2749 2762 ...               1               1             6.369
   343 426.0 3601 3628 ...               1               1 7.350049990155975
   344 231.0 3625 3644 

In [ ]:
highz_names = {
    "Guido": 0.8,
    "Sofia": 0.8,
    "Yeichi": 0.2,
    "Tommaso": 0.2,
}

lowz_names = {
    "Peter": 0.8,
    "Matt": 0.2,
    "Xianlong": 0.5,
    "Nicola": 0.5,
}
highz_bool = (cat_v2["V1_CLASS"] >= 4) & (
    ((cat_v2["REDSHIFT_USE"] >= 6) | (cat_v2["UNRELIABLE_REDSHIFT"] == True))
    & (np.isfinite(cat_v2["zphot"]))
)
highz_idxs = np.where(highz_bool)[0]

lowz_bool = (cat_v2["V1_CLASS"] >= 4) & (
    ((cat_v2["REDSHIFT_USE"] < 6) & (cat_v2["UNRELIABLE_REDSHIFT"] == False))
    | (~np.isfinite(cat_v2["zphot"]))
)
lowz_idxs = np.where(lowz_bool)[0]

import numpy as np

rng = np.random.default_rng()

cat_idxs = {}

cat_idxs["Matt"] = (lowz_bool & cat_v2["TENTATIVE_REDSHIFT"]) | np.array(
    [
        True if i in lowz_idxs[rng.random(len(lowz_idxs)) < 0.1] else False
        for i in np.arange(len(cat_v2))
    ]
)
cat_idxs["Tommaso"] = (highz_bool) & (
    cat_v2["TENTATIVE_REDSHIFT"]
    | (cat_v2["UNRELIABLE_REDSHIFT"] == False)
    | np.array(
        [
            True if i in highz_idxs[rng.random(len(highz_idxs)) < 0.15] else False
            for i in np.arange(len(cat_v2))
        ]
    )
)
cat_idxs["Peter"] = (
    (lowz_bool & cat_v2["TENTATIVE_REDSHIFT"])
    | np.array(
        [
            True if i in lowz_idxs[rng.random(len(lowz_idxs)) < 0.3] else False
            for i in np.arange(len(cat_v2))
        ]
    )
    # | (
    #     np.array(
    #         [
    #             True if i in highz_idxs[rng.random(len(highz_idxs)) < 0.2] else False
    #             for i in np.arange(len(cat_v2))
    #         ]
    #     )
    #     & (~cat_idxs["Tommaso"])
    # )
)
cat_idxs["Nicolo"] = (
    (~cat_idxs["Matt"])
    & (~cat_idxs["Peter"])
    & lowz_bool
    & np.array(
        [
            True if i in lowz_idxs[rng.random(len(lowz_idxs)) < 0.5] else False
            for i in np.arange(len(cat_v2))
        ]
    )
)
cat_idxs["Xianlong"] = (
    (~cat_idxs["Matt"]) & (~cat_idxs["Peter"]) & (~cat_idxs["Nicolo"]) & lowz_bool
)

cat_idxs["Yechi"] = (
    (~cat_idxs["Tommaso"])
    & (~cat_idxs["Peter"])
    & highz_bool
    & np.array(
        [
            True if i in highz_idxs[rng.random(len(highz_idxs)) < 0.33] else False
            for i in np.arange(len(cat_v2))
        ]
    )
)
cat_idxs["Guido"] = (
    (~cat_idxs["Tommaso"])
    & (~cat_idxs["Peter"])
    & (~cat_idxs["Yechi"])
    & highz_bool
    & np.array(
        [
            True if i in highz_idxs[rng.random(len(highz_idxs)) < 0.5] else False
            for i in np.arange(len(cat_v2))
        ]
    )
)
cat_idxs["Sofia"] = (
    (~cat_idxs["Tommaso"])
    & (~cat_idxs["Peter"])
    & (~cat_idxs["Yechi"])
    & (~cat_idxs["Guido"])
    & highz_bool
    # & np.array(
    #     [
    #         True if i in highz_idxs[rng.random(len(highz_idxs)) < 0.3] else False
    #         for i in np.arange(len(cat_v2))
    #     ]
    # )
)
# assigned = (
#     (~cat_idxs["Tommaso"]) & (~cat_idxs["Peter"]) & (~cat_idxs["Yechi"]) & highz_bool
# )
# not_assigned_idxs = np.where((~assigned) & highz_bool)[0]
# temp_g = np.array(
#     [
#         (
#             True
#             if i in not_assigned_idxs[rng.random(len(not_assigned_idxs)) < 0.5]
#             else False
#         )
#         for i in np.arange(len(cat_v2))
#     ]
# )
# print(np.nansum(assigned), np.nansum(temp_g))
# cat_idxs["Guido"] = assigned | temp_g
# cat_idxs["Sofia"] = (highz_bool) & ~temp_g

for k, v in cat_idxs.items():
    print(k, np.nansum(v))

lowz_array = np.array(
    [cat_idxs["Matt"], cat_idxs["Peter"], cat_idxs["Nicolo"], cat_idxs["Xianlong"]]
)
print(f"\nLow redshift:\nTotal:\t{len(lowz_idxs)}")
for v, c in zip(
    *np.unique(np.nansum(lowz_array[:, lowz_bool], axis=0), return_counts=True)
):
    print(f"Viewed {v} time(s): {c:0<3}")

highz_array = np.array(
    [
        cat_idxs["Tommaso"],
        cat_idxs["Yechi"],
        cat_idxs["Guido"],
        cat_idxs["Sofia"],
        cat_idxs["Peter"],
    ]
)
print(f"\nHigh redshift:\nTotal:\t{len(highz_idxs)}")
for v, c in zip(
    *np.unique(np.nansum(highz_array[:, highz_bool], axis=0), return_counts=True)
):
    print(f"Viewed {v} time(s): {c:0<3}")

Matt 171
Tommaso 182
Peter 296
Nicolo 197
Xianlong 195
Yechi 267
Guido 279
Sofia 288

Low redshift:
Total:	1024
Viewed 1 time(s): 880
Viewed 2 time(s): 144

High redshift:
Total:	1016
Viewed 1 time(s): 1016


In [ ]:
new_cat_dir = grizli_home_dir / "classification-stage-2" / "catalogues" / "input"
new_cat_dir.mkdir(exist_ok=True, parents=True)

for p, idxs in cat_idxs.items():
    if not (new_cat_dir / f"input_cat_{p}.fits").is_file():
        p_cat = cat_v2[idxs].copy()
        p_cat.write(new_cat_dir / f"input_cat_{p}.fits", overwrite=True)
        print (p, len(p_cat))

Matt 201
Tommaso 182
Peter 397
Nicolo 289
Xianlong 281
Yechi 267
Guido 279
Sofia 288


In [13]:
from grizli import fitting
from grizli.pipeline import auto_script

reextract_dir = Path.cwd() / "reextracted"
reextract_dir.mkdir(exist_ok=True, parents=True)
skipped_dir = Path.cwd() / "skipped"
skipped_dir.mkdir(exist_ok=True, parents=True)

for filetype in ["beams", "full", "1D", "row", "line", "log_par", "stack"]:
    (reextract_dir / filetype).mkdir(exist_ok=True, parents=True)
    (skipped_dir / filetype).mkdir(exist_ok=True, parents=True)

specz_dir = grizli_home_dir / "Extractions" / "specz"
photz_dir = grizli_home_dir / "Extractions" / "photz"

max_size = 450

for row in cat_v2[:]:
    obj_id = row["NUMBER"]
    obj_z = row["REDSHIFT_USE"]

    if row["V1_CLASS"] == 6:
        if not (skipped_dir / "full" / f"{root_name}_{obj_id:05}.full.fits").is_file():
            for filetype in ["beams", "full", "1D", "row", "line", "log_par", "stack"]:
                try:
                    [
                        shutil.copy2(p, (skipped_dir / filetype / p.name))
                        for p in (specz_dir / filetype).glob(
                            f"*{obj_id:05}.*{filetype}*"
                        )
                    ]
                except:
                    [
                        shutil.copy2(p, (skipped_dir / filetype / p.name))
                        for p in photz_dir.glob(f"*{obj_id:05}.*{filetype}*")
                    ]

    if not ((row["V1_CLASS"] == 4) | (row["V1_CLASS"] == 5)):
        continue

    if (reextract_dir / "full" / f"{root_name}_{obj_id:05}.full.fits").is_file():
        continue
    # # print (obj_id, obj_z, row["FLUX_AUTO"])
    # # continue
    # Maximum diagonal extent of detection bounding box, measured from centre
    # det_diag = np.sqrt((row["XMAX"] - row["XMIN"])**2 + (row["YMAX"] - row["YMIN"])**2)
    det_halfdiag = np.sqrt(
        (np.nanmax([row["XMAX"] - row["X"], row["X"] - row["XMIN"]])) ** 2
        + (np.nanmax([row["YMAX"] - row["Y"], row["Y"] - row["YMIN"]])) ** 2
    )

    # pixel scale is half detection
    # Include factor of 25% to account for blotting and pixelation effects
    est_beam_size = np.clip(
        int(np.nanmin([np.ceil(0.5 * 1.25 * det_halfdiag), max_size])),
        a_min=5,
        a_max=max_size,
    )

    pline = {
        "kernel": "square",
        "pixfrac": 1.0,
        "pixscale": 0.06,
        "size": int(np.clip(2 * est_beam_size * 0.06, a_min=3, a_max=30)),
        "wcs": None,
    }
    args = auto_script.generate_fit_params(
        pline=pline,
        field_root=root_name,
        min_sens=0.0,
        min_mask=0.0,
        include_photometry=False,  # set both of these to True to include photometry in fitting
        use_phot_obj=False,
    )

    try:
        try:
            mb = multifit.MultiBeam(
                str(specz_dir / "beams" / f"{root_name}_{obj_id:05}.beams.fits"),
                fcontam=0.2,
                min_sens=0.0,
                min_mask=0,
                group_name=root_name,
            )
        except:
            mb = multifit.MultiBeam(
                str(photz_dir / "beams" / f"{root_name}_{obj_id:05}.beams.fits"),
                fcontam=0.2,
                min_sens=0.0,
                min_mask=0,
                group_name=root_name,
            )
        # dir(mb)
        # print (mb.beams)
        # print (mb.PA)
        new_beam_list = []
        for filt, pa_dict in mb.PA.items():
            # print (filt, pa_dict)
            for pa, beam_idxs in pa_dict.items():
                if row[f"{filt},{pa}_USE"] == 1:
                    [new_beam_list.append(mb.beams[b]) for b in beam_idxs]

        # print (dir(new_beam_list[0]))
        # print (new_beam_list[0].direct.filter)
        # exit()
        new_mb = multifit.MultiBeam(
            new_beam_list, fcontam=0.2, min_sens=0.0, min_mask=0, group_name=root_name
        )
        # print (new_mb.beams[0].direct.filter)
        # for beam in new_beam_list:
        #     beam.direct.filter = beam.direct.filter.split("-")[-1]
        #     beam.grism.filter = beam.grism.filter.split("-")[-1]
        new_mb.write_master_fits()

        del mb, new_mb

        if np.isfinite(obj_z):
            zr = [obj_z * 0.8, obj_z * 1.2]
        else:
            zr = [0, 10.0]

        #     # mb.write_master_fits()
        # #     shutil.copy2(specz_dir / "beams" / f"{root_name}_{obj_id:05}.beams.fits", f"{root_name}_{obj_id:05}.beams.fits")
        # # except:
        # #     shutil.copy2(photz_dir / "beams" / f"{root_name}_{obj_id:05}.beams.fits", f"{root_name}_{obj_id:05}.beams.fits")
        print(f"Fitting {obj_id}...")

        _ = fitting.run_all_parallel(
            obj_id,
            zr=zr,
            dz=[0.01, 0.001],
            verbose=True,
            get_output_data=True,
            skip_complete=False,
            save_figures=True,
        )
        print("Fit complete, output saved.")
        for filetype in ["beams", "full", "1D", "row", "line", "log_par", "stack"]:
            [
                p.rename(reextract_dir / filetype / p.name)
                for p in Path.cwd().glob(f"*{obj_id}.*{filetype}*")
            ]
    except Exception as e:
        print(f"Extraction failed for {obj_id}.", e)

In [14]:
# from astropy.io import fits
# from astropy.table import Table, vstack, join
# from astropy.coordinates import match_coordinates_sky, SkyCoord
# from astropy.wcs import WCS
# import astropy.visualization as astrovis
# import astropy.units as u
# import matplotlib.pyplot as plt
# import numpy as np

# plot = True

# if not (catalogue_dir / "ASTRODEEP_photz.fits").is_file():

#     uncover_cat = Table.read(catalogue_dir / "UNCOVER_DR3_SPS_redshift_catalog.fits")

#     uncover_cat.rename_columns(["id", "z_ml"], ["id_uncover", "zphot"])

#     astrodeep_cat = Table.read(catalogue_dir / "ASTRODEEP-JWST-ABELL2744_photoz.fits")

#     astrodeep_cat.rename_columns(["ID", "RA", "DEC"], ["id_astrodeep_zphot", "ra", "dec"])
#     # alt_cat.rename_columns(["id", "z_ALT"], ["id_alt", "zspec"])
#     # max_offset = 0.1 * u.arcsec

#     # use_idx = sep2d > max_offset

#     # astrodeep_cat = astrodeep_cat

#     # combined_spec_cat = vstack(
#     #     [astrodeep_cat[astrodeep_cat["zspec"] > -99], alt_cat[use_idx]]
#     # )
#     grizli_zspec_cat = Table.read(catalogue_dir / "grizli_specz_matched.fits")

#     # print (np.nansum(np.isin(astrodeepgrizli_zspec_cat["id_astrodeep"])))

#     # grp_idx, grp_sep2d = grp.catalog.match_to_catalog_sky(uncover_cat)
#     grp_idx, grp_sep2d = grp.catalog.match_to_catalog_sky(astrodeep_cat)

#     astrodeep_cat["NUMBER"] = np.full_like(
#         astrodeep_cat["ra"].shape, -99, dtype=int
#     )
#     uncover_cat["NUMBER"] = np.full_like(
#         uncover_cat["ra"].shape, -99, dtype=int
#     )
#     # print (grp.catalog.colnames)

#     print(len(np.unique(grp_idx)))
#     print(grp_idx)
#     unique_matches = np.unique(grp_idx)
#     for un in unique_matches:
#         match_idxs = np.argwhere((grp_idx == un) & (grp_sep2d < 1 * u.arcsec))
#         if len(match_idxs) == 0:
#             continue
#         # print(grp_idx[match_idxs])
#         if len(match_idxs) >= 2:
#             match_idxs = match_idxs[np.argmin(grp_sep2d[match_idxs])]
#         astrodeep_cat["NUMBER"][match_idxs] = grp.catalog["NUMBER"][un]
#         # uncover_cat["NUMBER"][match_idxs] = grp.catalog["NUMBER"][un]

#     # grizli_cat = Table.read(f"{root_name}-ir.cat.fits")
#     del grizli_zspec_cat["zphot"]
#     grizli_zphot_cat = join(grizli_zspec_cat, astrodeep_cat["NUMBER", "id_astrodeep_zphot", "zphot"], keys="NUMBER", join_type="left")
#     plot=False
#     if plot:
#         with fits.open(
#             grizli_extraction_dir.parent / "Prep" / f"{root_name}-ir_drc_sci.fits"
#         ) as hdul:
#             direct_img = hdul[0].data.copy()
#             direct_wcs = WCS(hdul[0].header.copy())
#             del hdul

#         # mismatched_spec_cat = combined_spec_cat[
#         #     # (grp_sep2d>1*u.arcsec)
#         #     (grp_sep2d <= 1 * u.arcsec)
#         #     # & (grp_sep2d > 0.75 * u.arcsec)
#         # ]
#         # print(len(mismatched_spec_cat))

#         fig, ax = plt.subplots(
#             # figsize=(20, 15),
#             dpi=600,
#             subplot_kw={"projection": direct_wcs},
#         )
#         ax.imshow(
#             direct_img,
#             norm=astrovis.ImageNormalize(
#                 direct_img,
#                 stretch=astrovis.LogStretch(),
#                 interval=astrovis.PercentileInterval(99.9),
#             ),
#             cmap="binary",
#         )
#         xlim, ylim = ax.get_xlim(), ax.get_ylim()
#         ax.scatter(
#             grizli_zphot_cat["ra"],
#             grizli_zphot_cat["dec"],
#             transform=ax.get_transform("world"),
#             c=grizli_zphot_cat["zphot"],
#             s=1,
#             vmin=0.25,vmax=0.35, cmap="rainbow"
#         )
#         ax.set_xlim(xlim), ax.set_ylim(ylim)
# # else:
#     # combined_photz_cat = Table.read(catalogue_dir / "ASTRODEEP_phostz.fits")
#     grizli_zphot_cat.write(catalogue_dir / "grizli_photz_matched.fits", overwrite=True)
# else:
#     grizli_zphot_cat = Table.read(catalogue_dir / "grizli_photz_matched.fits")


In [15]:
# grizli_zphot_cat.sort("FLUX_AUTO", reverse=True)

# photz_dir = Path.cwd() / "photz"
# photz_dir.mkdir(exist_ok=True, parents=True)
# for filetype in ["beams", "full", "1D", "row", "line", "log_par", "stack"]:
#     (photz_dir / filetype).mkdir(exist_ok=True, parents=True)

# max_size = 450

# for row in grizli_zphot_cat[:]:
#     obj_id = row["NUMBER"]
#     obj_z = row["zphot"]

#     if (
#         grizli_extraction_dir / "specz" / "full" / f"{root_name}_{obj_id:05}.full.fits"
#     ).is_file() or (
#         photz_dir / "full" / f"{root_name}_{obj_id:05}.full.fits"
#     ).is_file():
#         continue
#     if obj_id==2409:
#         continue
#     # Maximum diagonal extent of detection bounding box, measured from centre
#     # det_diag = np.sqrt((row["XMAX"] - row["XMIN"])**2 + (row["YMAX"] - row["YMIN"])**2)
#     det_halfdiag = np.sqrt(
#         (np.nanmax([row["XMAX"] - row["X"], row["X"] - row["XMIN"]])) ** 2
#         + (np.nanmax([row["YMAX"] - row["Y"], row["Y"] - row["YMIN"]])) ** 2
#     )

#     # pixel scale is half detection
#     # Include factor of 25% to account for blotting and pixelation effects
#     est_beam_size = int(np.nanmin([np.ceil(0.5 * 1.25 * det_halfdiag), max_size]))

#     pline = {
#         "kernel": "square",
#         "pixfrac": 1.0,
#         "pixscale": 0.06,
#         "size": int(np.clip(2*est_beam_size*0.06, a_min=3, a_max=30)),
#         "wcs": None,
#     }
#     args = auto_script.generate_fit_params(
#         pline=pline,
#         field_root=root_name,
#         min_sens=0.0,
#         min_mask=0.0,
#         include_photometry=False,  # set both of these to True to include photometry in fitting
#         use_phot_obj=False,
#     )

#     try:
#         if not (photz_dir / "beams" / f"{root_name}_{obj_id:05}.beams.fits").is_file():
        
#             print (f"Fetching beams for {obj_id}...")
#             beams = grp.get_beams(
#                 obj_id,
#                 size=est_beam_size, 
#                 min_mask=0,
#                 min_sens=0,
#                 beam_id="A",
#             )
#             mb = multifit.MultiBeam(
#                 beams, fcontam=0.2, min_sens=0.0, min_mask=0, group_name=root_name
#             )
#             mb.write_master_fits()

#             print (f"Saved beams for {obj_id}.")
#         else:
#             shutil.copy2(photz_dir / "beams" / f"{root_name}_{obj_id:05}.beams.fits", f"{root_name}_{obj_id:05}.beams.fits")

#         print(f"Fitting {obj_id}...")

#         if np.isfinite(obj_z):
#             zr = [obj_z * 0.9, obj_z * 1.1]
#         else:
#             zr = [0, 5.0]

#         _ = fitting.run_all_parallel(
#             obj_id,
#             zr=zr,
#             dz=[0.01, 0.001],
#             verbose=True,
#             get_output_data=True,
#             skip_complete=False,
#             save_figures=True,
#         )
#         print("Fit complete, output saved.")
#         for filetype in ["beams", "full", "1D", "row", "line", "log_par", "stack"]:
#             [
#                 p.rename(photz_dir / filetype / p.name)
#                 for p in Path.cwd().glob(f"*{obj_id}.*{filetype}*")
#             ]
#     except Exception as e:
#         print(f"Extraction failed for {obj_id}: {e}")